In [1]:
import torch
from torch import optim
from torch.autograd import Variable
from torch.utils.data import DataLoader

In [2]:
import importlib
# import [module]
# importlib.reload([module])

In [ ]:
from custom.models import u_net
from custom.utils import conditioned_inverse
from custom.datasets import nuclei_dataset, nuclei_dataset_gen
from custom.loss import weighted_bce_loss_with_logits as loss_func

/root/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.16) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
nuclei_dataset_df = nuclei_dataset_gen(path='../data/stage1_train/')

/home/data/notebook/data/yujie/cell_segmentation/cell_detection/custom/datasets.py:100: RuntimeWarning: invalid value encountered in greater
  dist_large_value_idx = dist > 10
/home/data/notebook/data/yujie/cell_segmentation/cell_detection/custom/datasets.py:102: RuntimeWarning: overflow encountered in power
  weight = get_weight_c(mask) + np.where(dist_large_value_idx, .003, 10 * np.exp(-2 * np.power(dist, 2) / np.power(sigma, 2)))
/home/data/notebook/data/yujie/cell_segmentation/cell_detection/custom/datasets.py:94: RuntimeWarning: invalid value encountered in logical_or
  mask = np.logical_or(mask_, mask)
/home/data/notebook/data/yujie/cell_segmentation/cell_detection/custom/datasets.py:114: UserWarning: class_0 count zero!
  warnings.warn("class_0 count zero!")
/home/data/notebook/data/yujie/cell_segmentation/cell_detection/custom/datasets.py:102: RuntimeWarning: invalid value encountered in power
  weight = get_weight_c(mask) + np.where(dist_large_value_idx, .003, 10 * np.exp(-2 *

In [ ]:
train_loader = DataLoader(nuclei_dataset(nuclei_dataset_df),batch_size=8, shuffle=True)
test_loader = DataLoader(nuclei_dataset(nuclei_dataset_df, train=False), batch_size=8)

# train_loader = DataLoader(nuclei_dataset(main_path='../data/stage1_train/data.csv', transform=custom_transform, from_saved=True), 
#                          batch_size=8, shuffle=True)
# test_loader = DataLoader(nuclei_dataset(main_path='../data/stage1_train/data.csv', train=False, from_saved=True), batch_size=8)

In [ ]:
from custom.models import u_net
model = u_net()
model.cuda()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (input, target, weight) in enumerate(train_loader):
        input = Variable(input).cuda()
        target = Variable(target).cuda()
        weight = Variable(weight).cuda()
        optimizer.zero_grad()
        output = model(input)
        loss = loss_func(output, target, weight)
        loss.backward()
        train_loss += loss.data[0]
        optimizer.step()

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))

In [ ]:
from torchvision.utils import save_image
def test(epoch):
    model.eval()
    test_loss = 0
    for i, (input, target, weight) in enumerate(test_loader):
        input = Variable(input).cuda()
        target = Variable(target).cuda()
        weight = Variable(weight).cuda()
        output = model(input)
        test_loss += loss_func(output, target, weight).data[0]
        if i == 0:
            n = 8
            comparison = torch.cat([input[:n], output[:n]])
            save_image(comparison.data.cpu(),
                     'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [ ]:
epochs = 10
for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)
#
